In [18]:
import os
from dotenv import load_dotenv,find_dotenv
import warnings
warnings.filterwarnings('ignore')
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

####  LLMChain

In [19]:
load_dotenv(find_dotenv(),override=True)

True

In [20]:
chat_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4")

In [21]:
temp_sample=""" suggest only {count} tourists spots for {place}"""

In [22]:
prompt=PromptTemplate(input_variables=["count","place"],template=temp_sample)

In [23]:
llmchain = LLMChain(llm=chat_model, prompt=prompt)
print(llmchain.run({"count":5,"place":"india"}))

1. Taj Mahal, Agra: One of the Seven Wonders of the World, the Taj Mahal is a must-visit monument known for its stunning architecture. It is a symbol of love, built by Mughal Emperor Shah Jahan for his wife Mumtaz Mahal.

2. Jaipur, Rajasthan: Known as the Pink City, Jaipur is famous for its magnificent forts, palaces and rich heritage. Visit the Amber Fort, Hawa Mahal, and City Palace for a glimpse of royal history.

3. Kerala Backwaters: Enjoy the serene beauty of the backwaters with a stay on a traditional houseboat. Kerala is also known for its beautiful beaches, wildlife sanctuaries, and Ayurvedic treatments.

4. Varanasi, Uttar Pradesh: One of the oldest continuously inhabited cities in the world, Varanasi is a spiritual hub with numerous temples and the famous Ganges River where pilgrims come to wash away their sins.

5. Goa: Known for its beautiful beaches, Portuguese heritage, and vibrant nightlife, Goa is a perfect destination for relaxation and fun. Explore the historic chur

In [24]:
input_list = [
    {"count":2,"place":"delhi"},
    {"count":3,"place":"lucknow"},
    {"count":4,"place":"varanasi"},
    
]

In [25]:
llmchain.generate(input_list)

LLMResult(generations=[[ChatGeneration(text="1. Qutub Minar: This is the second highest brick minaret in the world, standing at an impressive 240 feet. The construction started in 1192 by Qutub-ud-din Aibak and was completed by his successor Iltutmish. It's a UNESCO World Heritage Site, known for its intricate carvings and verses from the Quran.\n\n2. Humayun’s Tomb: Humayun’s Tomb is another UNESCO World Heritage Site in Delhi. It is a splendid example of early Mughal architecture. Built in the mid-16th century by Haji Begum, the Persian-born senior wife of Humayun, the second Mughal emperor, this place is worth visiting for its magnificent architecture and the lush gardens.", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="1. Qutub Minar: This is the second highest brick minaret in the world, standing at an impressive 240 feet. The construction started in 1192 by Qutub-ud-din Aibak and was completed by his successor Iltutmish. It's a UNESCO Wor

In [26]:
llmchain.apply(input_list)

[{'text': '1. The Red Fort: This iconic landmark, built by Emperor Shah Jahan, is an epitome of Mughal architecture. It houses several museums highlighting the history of India.\n\n2. Qutub Minar: A UNESCO World Heritage Site, this 73-meter tall tower is the second tallest minar in India. It displays the architectural brilliance of the medieval period.'},
 {'text': '1. Bara Imambara: This is an important monument in Lucknow that showcases Mughal architecture. The complex includes the large Asfi mosque, the Bhul Bhulaiya (the labyrinth), and a bowli, a step well with running water. \n\n2. The Residency: This is a group of buildings that served as the residence for the British Resident General who was a representative in the court of the Nawab. The structure was heavily damaged in the first war of Independence in 1857 and now stands as a symbol of British colonialism in India.\n\n3. Rumi Darwaza: This is an impressive gateway which was built under the patronage of Nawab Asaf-Ud-Daula in 

In [27]:
llmchain.run(input_list[1])

'1. Bara Imambara: This is an architectural marvel in Lucknow, built by Asaf-ud-Daula, the Nawab of Lucknow, in 1784. Known for its incredible maze called Bhul Bhulaiya, it offers a panoramic view of the city from the top. \n\n2. Rumi Darwaza: Also known as Turkish Gate, it was built in the year 1784 by Nawab Asaf-ud-daula. The 60 feet tall gate is one of the finest examples of Awadhi architecture and is now an iconic symbol of Lucknow.\n\n3. The Residency: Known for its historical significance, the Residency is a group of buildings that were built in 1800 AD by the then Nawab of Oudh, Nawab Saadat Ali Khan. It was the site of the famous Siege of Lucknow during the Indian Rebellion of 1857.'

### Router Chains

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [29]:
doctor_temp="""consider you are healthcare expert , You are great at answering question realted to medical and healthcare.
             when you donot know the answer to  a question you admit that you donot know.
             here is a question:
             {input}
             """
data_scientist_temp="""consider you are data scientist and having expert in Deeplarning and NLP , You are great at answering question realted AI and data science.
             when you donot know the answer to  a question you admit that you donot know.
             here is a question:
             {input}
             """
agri_temp="""consider you are farmer and having expert in farming and agriculture , You are great at answering question realted farming.
             when you donot know the answer to  a question you admit that you donot know.
             here is a question:
             {input}
             """

In [30]:
prompt_infos = [
    {
        "name": "doctor",
        "description": "Good for answering questions about healthcare",
        "prompt_template": doctor_temp,
    },
    {
        "name": "data scientist",
        "description": "Good for answering Deeplearning amd AI questions",
        "prompt_template": data_scientist_temp,
    },
    {
        "name":"agriculture",
        "description":"Good for answering questions about Agriculture and Farming",
        "prompt_template":agri_temp
    }
]

In [31]:
destination_chains = {}
for prompt_info in prompt_infos:
    name=prompt_info["name"]
    prompt_template=prompt_info["prompt_template"]
    prompt=PromptTemplate(template=prompt_template,input_variables=["input"])
    chain=LLMChain(llm=chat_model,prompt=prompt)
    destination_chains[name]=chain

In [32]:
default_chain = ConversationChain(llm=chat_model, output_key="text")

In [33]:
default_chain.run("What is Cancer explain?")

"Cancer is a group of more than 100 different diseases characterized by the uncontrolled, abnormal growth of cells that can form a tumor or spread to other parts of the body. This process, known as metastasis, is a key feature of invasive cancers and is often the cause of death in cancer patients.\n\nCancer can start anywhere in the body and is categorized based on where it begins and the type of cells it affects. For example, lung cancer starts in the lungs, while breast cancer starts in the breast. The cells can become cancerous due to a variety of factors including genetic mutations, exposure to certain chemicals and substances, or certain behaviors like smoking or excessive sun exposure.\n\nCancer symptoms can vary widely depending on the type and stage of the disease. Some common symptoms may include fatigue, weight loss, pain that doesn't go away, skin changes, changes in bowel or bladder habits, unusual bleeding, persistent cough, and difficulty swallowing.\n\nIt's important to 

In [34]:
# Creating LLMRouterChain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

doctor: Good for answering questions about healthcare
data scientist: Good for answering Deeplearning amd AI questions
agriculture: Good for answering questions about Agriculture and Farming


In [35]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [36]:
# creating the router chain
router_chain = LLMRouterChain.from_llm(chat_model, router_prompt)

# Multiple Prompt Chain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [37]:
# test it out
print(chain.run("What is ai ?"))
print(chain.run("What is COVID-19 ?"))
print(chain.run("What is deeplearning"))
print(chain.run("which plant is good for farming in summer season"))



> Entering new MultiPromptChain chain...
data scientist: {'input': 'What is ai ?'}
> Finished chain.
AI, or Artificial Intelligence, refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning (the acquisition of information and rules for using the information), reasoning (using the rules to reach approximate or definite conclusions), and self-correction.

AI can be categorized as either weak or strong. Weak AI, also known as narrow AI, is an AI system that is designed and trained for a particular task. Virtual personal assistants, such as Apple's Siri, are a form of weak AI. 

Strong AI, also known as artificial general intelligence, is an AI system with generalized human cognitive abilities. When presented with an unfamiliar task, a strong AI system is able to find a solution without human intervention.

Technologies that enable AI to solve more complex tasks include machine learning, deep learning, natural lan